In [3]:
import json

with open("/home/hungpv/projects/memory_data/original_data/locomo10.json","r") as f:
    original_dataset = json.load(f)

In [ ]:
original_dataset[0]

{'qa': [{'question': 'When did Caroline go to the LGBTQ support group?',
   'answer': '7 May 2023',
   'evidence': ['D1:3'],
   'category': 2},
  {'question': 'When did Melanie paint a sunrise?',
   'answer': 2022,
   'evidence': ['D1:12'],
   'category': 2},
  {'question': 'What fields would Caroline be likely to pursue in her educaton?',
   'answer': 'Psychology, counseling certification',
   'evidence': ['D1:9', 'D1:11'],
   'category': 3},
  {'question': 'What did Caroline research?',
   'answer': 'Adoption agencies',
   'evidence': ['D2:8'],
   'category': 1},
  {'question': "What is Caroline's identity?",
   'answer': 'Transgender woman',
   'evidence': ['D1:5'],
   'category': 1},
  {'question': 'When did Melanie run a charity race?',
   'answer': 'The sunday before 25 May 2023',
   'evidence': ['D2:1'],
   'category': 2},
  {'question': 'When is Melanie planning on going camping?',
   'answer': 'June 2023',
   'evidence': ['D2:7'],
   'category': 2},
  {'question': "What is Car

In [6]:
original_dataset[0].keys()

dict_keys(['qa', 'conversation', 'event_summary', 'observation', 'session_summary', 'sample_id'])

In [7]:
import re
def find_evidence(dia_id):
    # print(dia_id)
    match = re.match(r"[A-Za-z]+(\d+):(\d+)", dia_id)
    if match:
        num1, num2 = match.groups()
        return (int(num1), int(num2))

In [28]:
import copy

processed_data = []
for data in original_dataset:
    conv_id = data['sample_id']
    # for qa in data['qas']:
    #     qas.append({
    #         "question": qa['question'],
    #         "answer": qa['answer'],
    #         "evidences": 
    qas = []
    for q in data['qa']:
        # if not q.get("answer"):
        #     print(q)
        # print(q)
        evidences = []
        for e in q['evidence']:

            e_id = find_evidence(e)
            # print(type(e_id))
            if not isinstance(e_id, tuple):
                # print('y')
                continue
            
            session_idx, turn_idx= e_id
            # print(1)
            # print(type(record['conversation'][f'session_{session_idx}']))
            # print(2)
            for text in data['conversation'][f'session_{session_idx}']:
                # print(f'hh {e} {text}')
                if e == text['dia_id']:
                    evidence_text = text['text']
                    break
            evidences.append(evidence_text)
        
        qas.append(
            {
                "question":q['question'],
                'answer':q.get('answer') or q.get("adversarial_answer"),
                'evidences':evidences,
                'category':q['category']
            }
        )
    dialogs = []
    for i in range(1,1000):
        if f'session_{i}' not in data['conversation']:
            break
        
        messages = []
        for message in data['conversation'][f'session_{i}']:
            messages.append(
                {
                    "role":message['speaker'],
                    "content":message['text']
                }
            )
        dialogs.append(
            {
                "session_id": f'session_{i}',
                "datetime": data['conversation'][f'session_{i}_date_time'],
                "messages":messages
            }
        )
    
    processed_data.append(
        {
            "conv_id":conv_id,
            "qas":qas,
            "dialogs": dialogs
        }
    )
    



        # break
    # print(qas)
    

In [34]:
with open("/home/hungpv/projects/memory_data/processed_data/locomo_processed_data.json",'w') as f:
    json.dump(processed_data,f,indent=4, ensure_ascii=False)